# Open Vswitch

This example shows how to deploy a topology of Openvswitch software bridges on FABRIC. 

### Background: OVS

Open vSwitch ([OVS](https://www.openvswitch.org/)) is an open-source, multi-layer virtual switch that is designed to be used in virtualized server environments. It is a critical component of many virtualization platforms and software-defined networking (SDN) solutions.


### Target FABRIC Topology

This notebook will create a topology depicting a network with a bridge node and two hosts, where OVS is used to create virtual bridge. The OVS bridge enables communication between the hosts. This setup provides a flexible and scalable architecture, suitable for various virtualization and software-defined networking scenarios.

A high level view of the topology is depicted in the figure below.

<img src="./figs/openvswitch-single.png" width="70%"><br>

### Limitation
Due to NVIDIA/Mellanox limitation, it appears the desired bridged virtual function forwarding behavior is not something currently supported for SRIOV functions i.e. `NIC_Basic`. VM hosting OVS bridge is required to use Dedicated NICs i.e. `NIC_ConnectX_5` or `NIC_ConnectX_6`.

We are working on an alternate solution, which would be available in the upcoming release.

## Import the FABlib Library

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import ipaddress
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager


fablib = fablib_manager()
fablib.show_config();

## Create the Experiment Slice

### General Configuration

Set the name of the slice, the sites to use, and the number of additional (non-router) nodes to add to each site.

In [ ]:
slice_name= "MySlice-openvswitch"

# we will use CX5 to generate traffic so need sites that have CX5 for this example.
cx5_column_name = 'nic_connectx_5_available'

# find two sites with available ConnectX-5 
sites = [site1] = fablib.get_random_sites(count=1, filter_function=lambda x: x[cx5_column_name] > 0)
print(f"Sites: {sites}")

site_node_count = 2
bridge1_name = 'bridge1'

### Create Slice

In [ ]:
slice = fablib.new_slice(name=slice_name)
default_image= 'default_ubuntu_22'

### Add Bridge1

In [ ]:
bridge1 = slice.add_node(name=bridge1_name, site=site1, cores=4, ram=8, disk=50, image=default_image)
bridge1_nic = bridge1.add_component(model='NIC_ConnectX_5', name='nic_local')

### Add Host Nodes

In [ ]:
i = 0
print(f"Adding nodes to {site1}")
for node_num in range(site_node_count):
    node_name = f"{site1.lower()}{node_num+1}"

    node = slice.add_node(name=node_name, site=site1, cores=4, ram=8, disk=50, image=default_image)
    iface = node.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]    
    net = slice.add_l2network(name=f"net{node_num}")

    net.add_interface(iface)
    net.add_interface(bridge1_nic.get_interfaces()[i])
    i += 1

### Submit the Slice Request

<img src="./figs/openvswitch-single.png" width="60%"><br>

Now that the slice request is complete, you can submit it.


In [ ]:
slice.submit();

### Install OVS and net-tools on all bridges

In [ ]:
try:
    for node in slice.get_nodes():
        if node.get_name().startswith("bridge"):
            stdout, stderr = node.execute('yes | sudo apt-get -y update && sudo apt-get upgrade', quiet=True) 
            stdout, stderr = node.execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common', quiet=True)
            stdout, stderr = node.execute('sudo apt-get -y install net-tools', quiet=True)
            print(f"done bridge: {node.get_name()}")
    print("Done")
except Exception as e:
    print(f"Exception: {e}")

### Create a new bridge, enable the spanning tree protocol on necessary interfaces

In [ ]:
bridge1 = slice.get_node(name=bridge1_name)
stdout, stderr = bridge1.execute('sudo ovs-vsctl add-br br0')
for interface in bridge1.get_interfaces():
    stdout, stderr = bridge1.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
    #Remove IP addresses for all interfaces
    stdout, stderr = bridge1.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
#bring the bridge up
stdout, stderr = bridge1.execute('sudo ifconfig br0 up')

print("Done")

### Enable Spanning tree and confirm

In [ ]:
stdout, stderr = bridge1.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')

In [ ]:
stdout, stderr = bridge1.execute('sudo ovs-appctl stp/show')

## Setup the Host Nodes

In [ ]:
host1 = slice.get_node(name=f'{site1.lower()}1')
host2 = slice.get_node(name=f'{site1.lower()}2')

In [ ]:
stdout, stderr = host1.execute('sudo apt-get -y install net-tools', quiet=True)
stdout, stderr = host2.execute('sudo apt-get -y install net-tools', quiet=True)

In [ ]:
stdout, stderr = host1.execute(f'sudo ip link set dev {host1.get_interfaces()[0].get_physical_os_interface_name()} up', quiet=True)
stdout, stderr = host2.execute(f'sudo ip link set dev {host2.get_interfaces()[0].get_physical_os_interface_name()} up', quiet=True)

In [ ]:
stdout, stderr = host1.execute(f'sudo ip addr add 10.10.10.1/24 dev {host1.get_interfaces()[0].get_physical_os_interface_name()}', quiet=True)
stdout, stderr = host2.execute(f'sudo ip addr add 10.10.10.2/24 dev {host2.get_interfaces()[0].get_physical_os_interface_name()}', quiet=True)

### Ping Test

In [ ]:
# Ping test
stdout, stderr = host1.execute('ping 10.10.10.2 -c 5')

### Delete the slice

In [ ]:
slice.delete()